In [ ]:
# Import Libraries
import json
import os
from openai import AzureOpenAI

In [ ]:
# Set the environment variables
os.environ["AZURE_OPENAI_API_KEY"] = "REPLACE_WITH_YOUR_KEY_VALUE"
os.environ["AZURE_OPENAI_ENDPOINT"] = "REPLACE_WITH_YOUR_ENDPOINT"

In [ ]:
# Create a client object with preview version
client = AzureOpenAI(
 azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
 api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
 api_version="2024-03-01-preview"
)

In [ ]:
# Function to convert meters to inches
def convert_meters_to_inches(meters):
    """Convert meters to inches"""
    inches = round(meters * 39.37, 3)  # 1 meter is approximately 39.37 inches
    return json.dumps({"meters": meters, "inches": inches})

In [ ]:
# Example usage of the function
meters_input = 1.5  # Example input in meters
conversion_result = convert_meters_to_inches(meters_input)
print(f"Conversion result: {conversion_result}")

In [ ]:
# Set System Message
system_message = """You are a helpful scientific assistant."""

In [ ]:
user_message = """The distance from the Earth to the moon is 
roughly 384,000,000 meters. What is this distance in inches?"""

In [ ]:
# Set your model deployments
deployment_4 = "your-gpt-4-model"
deployment_3_turbo = "your-gpt-3.5-turbo-model"

In [ ]:
# Define messages
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

In [ ]:
# Create System and User Messages
system_message = """You are a translation assistant. A user will provide you with a text to translate between triple pluses
(+++), a target language between triple stars (***), and context 
around the translation between triple ampersands (&&&)."""

user_message = """+++Dear customer, we regret to inform that the 
item you ordered is currently-out-of-stock.+++ ***Spanish*** 
&&&We are trying to send detailed yet polite e-mails to 
customers.&&&"""

In [ ]:
# Create a tool object
tools = [
    {"type": "function",
     "function": {
         "name": "convert_meters_to_inches",
         "description": "Converts meters to inches",
         "parameters": {
             "type": "object",
             "properties": {
                 "meters": {
                     "type": "number",
                     "description": "The length in meters to be converted to inches"
                 }
             },
             "required": ["meters"]
         }
     }
    }
]


In [ ]:
# Obtain Response
response = client.chat.completions.create(
    model=deployment_4,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

In [ ]:
# Access the first item in the list to get the tool
func = response.choices[0].message.tool_calls[0]

# Get the function name
function_name = func.function.name

# Parse the JSON in the 'arguments' and extract the 'meters' value
arguments = json.loads(func.function.arguments)
meters = arguments["meters"]

In [ ]:
# Pass the function and obtain answers
func_to_call = globals()[function_name]
func_to_call(meters)